# Steps
1. Set Translation Language
2. upload one json to translate
3. run all cells
4. download your translation


# TODO:
##Fancier version will:
1. process any number of files
2. connect open-ai
3. connect gguf
4. remove any \u00e8 type characters


...

for writeup:
discuss balance of system-1 and system -2
remove any \u00e8 type characters




In [1]:
target_language = "italian"

json translator tool
auto-translate the 'value' fields in a json

Steps:

initial input:
("translate_this_json" = 'JSON TEXT',
"translate_into_this_language" = 'NAME OF LANGUAGE')

Save original json:
	-> original_translate_this_json

Save draft of final json:
keys from translate_this_json ->
	-> final_translated_json_draft with empty string values

Map Json key structure/format:
	-> required_json_key_format

Make master dictionary of lists of translations:
- convert to python dict ->  dict_of_lists_of_translations

X times, batch translation of json:
- translation: translate original_translate_this_json values
   -> translate_into_this_language
   (translate the whole json in one batch
- confirm JSON structure of output
  (if not correct SON structure, loop back repeat translation
- store each in dict_of_lists_of_translations

Rank Quality of Translations:
	- rank each a list in dict_of_lists_of_translations

	- make a ranked list
	- pick the first one
	- store in final_translated_json_draft

Make Final Translation
final_translated_json = final_translated_json_draft

Output
output_body = {
'translate_this_json': translate_this_json
'dict_of_lists_of_translations': dict_of_lists_of_translations
'final_translated_json_draft': final_translated_json_draft
}

final output:
	return output_body


Version:
1. python script
2. endpoint
3. using api-model service (online)
4. using local .GGUF model (offline)
	- make:
		A. raw llama.cpp wrapper
		B. call_api for python-lamma-cpp




In [2]:
"""
.env: get your environment variables:
  Using the Google Secretes (like.env) system
  built into colab on the left menu: the 'key' icon.
"""
from google.colab import userdata
mistral_api_key = userdata.get('mistral_api_key')
# mistral_api_key = 'xxx'

In [3]:
# Select Model



######################
# Mixtral 8x7 "Small"
######################
use_this_model = "mistral-small"

######################
# Mixtral Large ???
######################
use_this_model = "mistral-large-latest"

####################
# Mistral 7b "Tiny"
####################
use_this_model = "mistral-tiny"

# make a list of json files

In [ ]:
import os

def list_json_files_in_cwd():
    """
    Returns a list of .json files in the current working directory.
    """
    # List all files in the current working directory
    files_in_cwd = os.listdir('.')

    # Filter the list to include only .json files
    json_files = [file for file in files_in_cwd if file.endswith('.json')]

    return json_files

# Example usage
json_files_list = list_json_files_in_cwd()
print("JSON files in the CWD:", json_files_list)

# Process Json
- make a model/skeleton template

In [ ]:
import json

def load_json_file(file_path):
    """
    Loads and reads a JSON file, returning its content as a Python dictionary.

    :param file_path: Path to the .json file to be read.
    :return: A dictionary containing the data from the JSON file.
    """
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

def create_deep_empty_structure(data):
    """
    Recursively creates a deep copy of the provided data structure,
    replacing all terminal values with empty lists, and maintaining the same structure.

    :param data: The original data structure, potentially containing nested dictionaries.
    :return: A new data structure with the same keys but with empty lists as terminal values.
    """
    if isinstance(data, dict):  # Checks if the current data item is a dictionary
        return {key: create_deep_empty_structure(value) for key, value in data.items()}
    else:
        return []  # Replaces non-dictionary values with an empty list

def create_empty_json_file(original_data, new_file_path):
    """
    Creates a new JSON file replicating the structure of the original data,
    but with empty lists for all terminal values.

    :param original_data: Dictionary containing the original data.
    :param new_file_path: Path for the new .json file to be created.
    """
    # Create a new structure with the same keys but with empty lists as terminal values
    empty_data = create_deep_empty_structure(original_data)

    # Write this new structure to a new JSON file
    with open(new_file_path, 'w') as file:
        json.dump(empty_data, file, indent=4)

    return empty_data

# Assuming 'json_files_list' is defined and contains paths to your JSON files
# Example usage
original_file_path = json_files_list[0]  # Path to the original JSON file
new_file_path = 'new_empty_lists.json'  # Path for the new JSON file

# Load the original JSON file
original_data = load_json_file(original_file_path)

# Create a new JSON file with the deep empty structure
skeleton_json = create_empty_json_file(original_data, new_file_path)
print(f"New JSON file created at: {new_file_path}")
print(f"New skeleton_json: {skeleton_json}")




# put translation into list_skeleton_json

In [6]:
def list_skeleton_json_with_data(skeleton_json, source_data):
    """
    Recursively fills the skeleton_json structure with data from the source_data.

    :param skeleton_json: The skeleton_json structure with empty lists as terminal values.
    :param source_data: The source data structure from which values are to be copied.
    :return: The skeleton_json structure filled with values from source_data.
    """
    if isinstance(skeleton_json, dict) and isinstance(source_data, dict):
        for key in skeleton_json.keys():
            if key in source_data:
                skeleton_json[key] = list_skeleton_json_with_data(skeleton_json[key], source_data[key])
    elif isinstance(skeleton_json, list) and not skeleton_json:  # Check if skeleton_json is an empty list
        return source_data  # Replace the empty list with the source data value
    return skeleton_json


## save json utility

In [26]:

from datetime import datetime as dt

def save_json_to_file(input_file, file_path, optional_tag=""):
    """
    Saves a JSON object to a file.

    :param data: The JSON object to save.
    :param file_path: The path to the JSON file where the object should be saved.
    """
    # with open(file_path, 'w') as file:
    #     json.dump(data, file, indent=4)

    # make readable time
    date_time = dt.utcnow()
    clean_timestamp = date_time.strftime('%Y%m%d%H%M%S%f')


    new_title = f"{target_language}_{clean_timestamp}_{file_path}"

    new_title = optional_tag + new_title

    with open(new_title, 'w') as outfile:
        json.dump(input_file, outfile, indent=4)

# Functions to call Mistral

In [8]:
"""
related tools:

api throttle_protection
api async

import requests
"""
import requests
import json
import os
import re

from google.colab import userdata


mistral_api_key = userdata.get('mistral_api_key')

# Define the endpoint URL
endpoint_url = "https://api.mistral.ai/v1/chat/completions"

# Set the headers
headers = {
  "Content-Type": "application/json",
  "Accept": "application/json",
  "Authorization": f"Bearer {mistral_api_key}"
}

"""
# mode: [{"role": "user", "content": "say yes"}]

    # Define the request body
    request_body = {
      "model": "mistral-small",
      "messages": [{"role": "user", "content": user_input}]
    }

    # Send the request
    response = requests.post(endpoint_url, headers=headers, json=request_body)
"""




def add_to_context_history(role, comment):

    if role == 'user':
        segment = {"role": "user", "content": comment}

    elif role == 'assistant':
        segment = {"role": "assistant", "content": comment}

    elif role == 'system':
        segment = {"role": "system", "content": comment}

    else:
        print("add_to_context_history(role, comment)")
        print(role, comment)
        print('error')

    return segment


def prompt_user(user_input, context_history):

    context_history.append( add_to_context_history("user", user_input) )

    return context_history


def one_response_to_user(user_input, context_history, use_this_model):
    """
    Input: context_history
    Ouput Tuple: assistant_says, context_history
    """

    # prompt user
    context_history = prompt_user(user_input, context_history)

    # prompt assistant
    response = ask_mistral_tiny(context_history, use_this_model)

    return response


def ask_mistral_tiny(context_history, use_this_model):
    # Define the request body
    request_body = {
      "model": use_this_model,
      "messages": context_history
    }

    #################
    #################
    # Hit the ai api
    #################
    #################
    # Send the request
    response = requests.post(endpoint_url, headers=headers, json=request_body)

    # Check the response status code
    if response.status_code != 200:
        raise Exception(f"Error: {response.status_code} {response.text}")


    # Get the response data
    response_data = response.json()

    # Print the Mistral response

    ##
    ##
    # Turn this print on to see full return data
    ##
    ##
    """
    e.g.
    {
      "id": "635cb8d445ujhe5546bb64e5e7",
      "object": "chat.completion",
      "created": 170hrjfjf7084,
      "model": "mistral-tiny",
      "choices": [
        {
          "index": 0,
          "message": {
            "role": "assistant",
            "content": "Enjoy your cup of tea!"
          },
          "finish_reason": "stop",
          "logprobs": null
        }
      ],
      "usage": {
        "prompt_tokens": 575,
        "total_tokens": 629,
        "completion_tokens": 54
      }
    }
    """
    # print(json.dumps(response_data, indent=2))
    # print(type(response_data))

    output = response_data
    # print(type(output))
    # print(type(output["choices"][0]))

    # extract just the 'what they said' part out
    assistant_says = output["choices"][0]['message']['content']


    return assistant_says


# Reset context history (input to ai)

In [9]:
# conversation-history setup
context_history = []

## Add System instructions for rules

In [10]:
################
# System Prompt
################

example_1 = {
"Form_One": {
"FirstName":"*First Name*",
"LastName":"*Last Name*",
"Email":"*Email*",}
}
example_2 = {
"Form_One": {
"FirstName":"TRANSLATED_First_Name",
"LastName":"TRANSLATED_Last Name",
"Email":"TRANSLATED_Email",}
}


# set translation language and structure of output in system
comment = f"""
You are a helpful translator bot that produces high
quality professional translations in precise json formats.

You only translate the leaf values or terminal values in the json.
e.g. If the original json says:
{example_1}
Only translate the final leaf values, NOT the keys above!
{example_2}

You only translate into {target_language}.
You only produce json output in exactly this structure {skeleton_json}.
Your translations are clear, accurate, helpful, honrable, brief, polite, and professional.
Your do you best to tranlsate every leaf value field leaving nothing blank.

You always double check your work and make sure the translation is
excellent in the context of the whole body of translation.
"""
role = "system"

context_history.append( add_to_context_history(role, comment) )

# print(context_history)

## Add 'user' request for translation

In [11]:
###########################
# User Translation Request
###########################

# set translation language and structure of output in system
comment = f"""

The person we are trying to help needs a json file
translated into [{target_language}].
Let's help them translate the values in their json file.

Carefully translate this original file into {target_language}
{original_data}.

Double check your work and make sure the translation is
accurate, brief, and polite.

Make sure your translation is expressed using valid json syntax,
as:
```json
JSON_OBJECT(S)
```

(e.g. no trailing delimiter, escape conflicting characters, etc).
The structure of your translation must fit this structure
exactly: {skeleton_json}
"""
role = "user"

context_history.append( add_to_context_history(role, comment) )

# print(context_history)

In [12]:

########################
# Check Json Formatting
########################

# Helper Function
def check_function_description_keys(dict_str, model_dict):
    """
    This function CAN fail and should fail
    if the AI needs to retry at a task.
    Do not stop server when this this triggers an exception.

    edge case: before there is a populated output_log

    if passing, this function will return a valid json object
    """

    """
    Extracts JSON string enclosed between ```json and ``` markers.

    Parameters:
    - text (str): The input text containing the JSON block.

    Returns:
    - str: The extracted JSON string, or an empty string if no JSON block is found.
    """

    try:
        pattern = r'```json\n([\s\S]*?)\n```'
        match = re.search(pattern, dict_str)
        dict_str =  match.group(1) if match else ''

    except Exception as e:
        print(f"Dictionary clean failed: {e}")
        return False

    # clean
    try:
        # try safety cleaning
        dict_str = dict_str.replace("True", "true")
        dict_str = dict_str.replace("False", "false")
        dict_str = dict_str.replace("None", "null")

        # # This conflicted with free language in description section...
        # dict_str = dict_str.replace("'", '"')

        # remove trailing delimiter comma
        print(f"{dict_str[:-6]}")
        dict_str = dict_str.replace('",\n}', '"\n}')

    except Exception as e:
        print(f"Dictionary clean failed: {e}")
        return False

    # load
    try:
        # try converting
        dict_str = json.loads(dict_str)

    except Exception as e:
        print(f"Dictionary load failed: {e}")
        return False


    # check if keys are the same
    try:
        result = dict_str.keys() == model_dict.keys()
        if result is False:
            print(f"Failed: keys are not the same.")
            return False

    except Exception as e:
        print(f"Failed with Exception: keys are not the same: {e}")
        return False

    # if ok...
    return dict_str

# # test
# dict_str = assistant_says
# jsonchecked_translation = check_function_description_keys(dict_str, skeleton_json)

# Check json structure against original

In [ ]:
retry_counter = 0
json_ok_flag = False




while not json_ok_flag:
    # get a translation
    dict_str = ask_mistral_tiny(context_history, use_this_model)

    # # check json structure
    jsonchecked_translation = check_function_description_keys(dict_str, skeleton_json)

    if jsonchecked_translation:
        json_ok_flag = True

    else:
        retry_counter += 1


print(f"retry_counter -> {retry_counter}")
jsonchecked_translation





# Save File

In [14]:

from datetime import datetime as dt

# make readable time
date_time = dt.utcnow()
clean_timestamp = date_time.strftime('%Y%m%d%H%M%S%f')


new_title = f"{target_language}_{clean_timestamp}_{json_files_list[0]}"

with open(new_title, 'w') as outfile:
    json.dump(jsonchecked_translation, outfile, indent=4)

# Put Flesh on the Skeleton
- add translations to the lists

In [ ]:
source_data = jsonchecked_translation

populated_skeleton = list_skeleton_json_with_data(skeleton_json, source_data)

populated_skeleton

In [16]:
context_history = []

In [17]:
################
# System Prompt to pick best traslation
################

example_1 = {
"Form_One": {
"FirstName":[],
"LastName":[],
"Email":[],}
}
example_2 = {
"Form_One": {
"FirstName":"TRANSLATED_First_Name",
"LastName":"TRANSLATED_Last Name",
"Email":"TRANSLATED_Email",}
}



# set translation language and structure of output in system
comment = f"""
You are a helpful translator bot that selects high
quality professional translations from a list and produces or reproduces precise json formats.


You select the one best translate in the {target_language}.
You only produce json output in exactly this structure {skeleton_json},
but with a leaf node text not a list.


e.g. The list option json will have this form:
{example_1}
And your final form must contain one best translation, not a list.
{example_2}


Your final translation selections are clear, accurate, helpful, honrable, brief, polite, and professional.
Your do you best to tranlsate every leaf value field leaving nothing blank.

You always double check your work and make sure the translation is
excellent in the context of the whole body of translation.
"""
role = "system"

context_history.append( add_to_context_history(role, comment) )

# print(context_history)

In [18]:
###########################
# User Translation Request to select best translation
###########################

# set translation language and structure of output in system
comment = f"""

The person we are trying to help
selecting the best {target_language} language translation for their final translated json file.
Let's help them select the best {target_language} language translation leaf node values for their json file.

Task to do: Carefully the best {target_language} language translation leaf node values
from this list_of_translations json {populated_skeleton}.

Double check your work and make sure the translation is
accurate, brief, and polite.

Make sure your translation is expressed using valid json syntax,
as:
```json
JSON_OBJECT(S)
```

(e.g. no trailing delimiter, escape conflicting characters, etc).
The structure of your translation must fit this structure
{skeleton_json} but with a single best translation not a list.
"""
role = "user"

context_history.append( add_to_context_history(role, comment) )

# print(context_history)

In [ ]:
print(context_history)

# Check json structure against original

In [ ]:
retry_counter = 0
json_ok_flag = False




while not json_ok_flag:
    # get a translation
    dict_str = ask_mistral_tiny(context_history, use_this_model)

    # # check json structure
    selected_translation_json = check_function_description_keys(dict_str, skeleton_json)

    if selected_translation_json:
        json_ok_flag = True

    else:
        retry_counter += 1


print(f"retry_counter -> {retry_counter}")
selected_translation_json





In [ ]:
selected_translation_json


In [27]:
save_json_to_file(selected_translation_json, json_files_list[0], "selected_")